In [ ]:
%%bash
cd /content/Robust-MSCN
git add .
git commit -m "Update notebook"
git push origin eric

[eric d90baea] Update notebook
 4 files changed, 8 insertions(+), 5 deletions(-)
 rewrite Robust-MSCN.ipynb (65%)


To github.com:Erostrate9/Robust-MSCN.git
   5e60cd9..d90baea  eric -> eric


# Install Enviroment

## Download workload queries (dataset)
Note: there're some typos in downloading scripts. They didn't download the imdb_data.json correctly. We have to correct download_imdb_workload.sh, download_job_workload.sh and download_imdb_updates.sh:
```bash
## downloading data about imdb schema etc.
wget -O imdb_data.json https://www.dropbox.com/s/nxtt17s4gdt21r5/imdb_data.json?dl=1
cp imdb_data.json queries/ceb-imdb-full/dbdata.json

```

In [ ]:
%%bash
# download IMDB workload
cd Robust-MSCN
bash scripts/download_imdb_workload.sh

## PostgreSQL Configuration

- We use docker to install and configure PostgreSQL, and setup the relevant databases.

- Make sure that you have Docker installed appropriately for your system, with the docker daemon running.
- PostgreSQL requires a username, which we copy from an environement variable \$LCARD_USER while setting it up in docker. Similarly, set \$LCARD_PORT to the local port you want to use to connect to the PostgreSQL instance running in docker. Here are the commands to set it up:

- You cannot run docker daemon on Colab (at least not on the free version). Therefore, I recommend running docker locall

- You can run the following bash on your local environment (e.g., Mac) to build the docker image
```bash
cd docker
export LCARD_USER=imdb
export LCARD_PORT=5432
docker build --build-arg LCARD_USER=imdb -t pg12 .
docker save -o pg12.tar pg12
docker run -itd --shm-size=1g --name card-db -p 5432:5432 -d pg12
docker restart card-db
docker exec -it card-db /imdb_setup.sh
```
- Alternatively, you can use image that I uploaded:
```bash
docker pull erostrate9/pg12:latest
```


- To build Docker image on Mac, you need to add the clang dependency in Dockfile.

```
RUN apt-get update && apt-get install -y \
    clang \
    wget \
    tar \
    git \
    gcc \
    build-essential \
    postgresql-server-dev-12
```

```
# if your instance gets restarted / docker image gets shutdown
sudo docker restart card-db

# get a bash shell within the docker image
sudo docker exec -it card-db bash
# note that postgresql data on docker is stored at /var/lib/postgresql/data

# If you failed to run `docker exec -it card-db /imdb_setup.sh`
/imdb_setup.sh



# connect psql on your host to the postgresql server running on docker
psql -d imdb -h localhost -U ceb
```


## Evaluating estimates
Given a query, and estimates for each of its subplans, we can use various error functions to evaluate how good the estimates are. We can directly compare the true values and the estimated values, using for instance:

Q-Error, Relative Error, Absolute Error etc. Q-Error is generally considered to be the most useful of these metrics from the perspective of query-optimization.
Alternatively, we can compare how good was the plan generated by using the estimated values. This will depend on the query optimizer - in particular the properties of the cost model we choose, and the search function etc. We provide implementations for the two options as discussed in the paper, but by changing configurations of the PostgreSQL cost model, or adding more complex custom cost models, there can be many possibilities considered here.

Postgres Plan Cost (PPC): this uses the PostgreSQL cost model with two restrictions --- no materialization and parallelism. For experimenting with different configurations, check the function set_cost_model in losses/plan_loss.py and add additional configurations.

Plan-Cost: this considers only left deep plans, and uses a simple user specified cost function (referred to as C in the paper).

Here is a self contained example showing the API to compute these different kind of errors on a single query.

In [ ]:
from os.path import dirname, join, realpath
import sys
sys.path.append('/content/Robust-MSCN')
import os
os.chdir('/content/Robust-MSCN')
import query_representation
from query_representation.query import *
from evaluation.eval_fns import *
from psycopg2 import OperationalError

qfn = join(dirname(realpath(query_representation.__path__._path[0])), "queries/joblight_train/joblight-train-all/0.pkl")
qrep = load_qrep(qfn)
ests = get_postgres_cardinalities(qrep)

# estimation errors for each subplan in the query
qerr_fn = get_eval_fn("qerr")
abs_fn = get_eval_fn("abs")
rel_fn = get_eval_fn("rel")

qerr = qerr_fn.eval([qrep], [ests], result_dir=None)
abs_err = abs_fn.eval([qrep], [ests], result_dir=None)
relerr = rel_fn.eval([qrep], [ests],result_dir=None)

print("avg q-error: {}, avg abs-error: {}, avg relative error: {}".format(
              np.round(np.mean(qerr),2), np.round(np.mean(abs_err), 2),
                            np.round(np.mean(relerr), 2)))

# check the function comments to see the description of the arguments
# can change the db arguments appropriately depending on the PostgreSQL
# installation.
ppc = get_eval_fn("ppc")
port = 5432
# try:
ppc = ppc.eval([qrep], [ests], user="imdb", pwd="password", db_name="imdb",
        db_host="localhost", port=port, num_processes=-1, result_dir=None,
        cost_model="cm1")
# except OperationalError:
#     print(f"PGSQL not running at port {port}")
#     pass

# we considered only one query, so the returned lists have just one element
print("PPC is: {}".format(np.round(ppc[0])))

pc = get_eval_fn("plancost")
plan_cost = pc.eval([qrep], [ests], cost_model="C")
print("Plan-Cost is: {}".format(np.round(plan_cost[0])))

avg q-error: 2.13, avg abs-error: 901816.17, avg relative error: 0.5
PPC is: 867303.0
, #samples: 1, Relative Plan Cost: 1.0
Plan-Cost is: 987052.0


### True cardinality

In [3]:
from os.path import dirname, join, realpath
import query_representation
from query_representation.query import *
from evaluation.eval_fns import *
from psycopg2 import OperationalError

qfn = join(dirname(realpath(query_representation.__path__._path[0])), "queries/joblight_train/joblight-train-all/0.pkl")
qrep = load_qrep(qfn)
ests = get_true_cardinalities(qrep)

# estimation errors for each subplan in the query
qerr_fn = get_eval_fn("qerr")
abs_fn = get_eval_fn("abs")
rel_fn = get_eval_fn("rel")

qerr = qerr_fn.eval([qrep], [ests], result_dir=None)
abs_err = abs_fn.eval([qrep], [ests], result_dir=None)
relerr = rel_fn.eval([qrep], [ests],result_dir=None)

print("avg q-error: {}, avg abs-error: {}, avg relative error: {}".format(
              np.round(np.mean(qerr),2), np.round(np.mean(abs_err), 2),
                            np.round(np.mean(relerr), 2)))

# check the function comments to see the description of the arguments
# can change the db arguments appropriately depending on the PostgreSQL
# installation.
ppc = get_eval_fn("ppc")
port = 5432
# try:
ppc = ppc.eval([qrep], [ests], user="imdb", pwd="password", db_name="imdb",
        db_host="localhost", port=port, num_processes=-1, result_dir=None,
        cost_model="cm1")
# except OperationalError:
#     print(f"PGSQL not running at port {port}")
#     pass

# we considered only one query, so the returned lists have just one element
print("PPC is: {}".format(np.round(ppc[0])))

pc = get_eval_fn("plancost")
plan_cost = pc.eval([qrep], [ests], cost_model="C")
print("Plan-Cost is: {}".format(np.round(plan_cost[0])))

avg q-error: 1.0, avg abs-error: 0.0, avg relative error: 0.0
PPC is: 510222.0
, #samples: 1, Relative Plan Cost: 1.0
Plan-Cost is: 987052.0


## Getting runtimes
- There are two steps to generating the runtimes; - first, we generate the Postgres Plan Cost, and the corresponding SQLs to execute.
- These SQL strings would be annotated with various pg_hint_plan hints to enforce join order, operator selection and index Postgres Plan Costselection (see losses/plan_losses.py for details). These strings can be executed on PostgreSQL with pg_hint_plan loaded, but you may want to use a different setup for execution --- so other processes on the computer do not interfere with the execution times, and do things like clear the cache after every execution (cold start), or repeat each execution a few times etc. depending on your goals.
- Here, we provide a simple example to execute the SQLs, but note that this does not clear caches, or take care about isolating the execution from other processes, so these timings won't be reliable.
```python
# writes out the file results/Postgres/PostgresPlanCost.csv with the sqls to execute
# the sqls are modified with pg_hint_plan hints to use the cardinalities output
# by the given algorithm' estimates;
python3 main.py --algs postgres -n 5 --query_template 1a --eval_fns qerr,ppc,plancost

# executes the sqls on PostgreSQL server, with the given credientials
python3 evaluation/get_runtimes.py --port 5432 --user ceb --pwd password --result_dir results/Postgres
```

## Learned Model

In [1]:
import wandb
# log in wandb to upload/download weights & Biases
# https://wandb.ai/authorize
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erostrate9 (ericsun). Use `wandb login --relogin` to force relogin


True

In [ ]:
%%bash
cd /content/Robust-MSCN
bash scripts/download_imdb_workload.sh

### Seen template

In [16]:
!python3 main.py --config configs/config-seen-postgres.yaml --alg true --result_dir ./results/seen/true

device used in experiemnt: cpu
flow loss C library not being used as we are not on linux
data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: query
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: 16
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like

In [15]:
!python3 main.py --config configs/config-seen-postgres.yaml --alg postgres --result_dir ./results/seen/postgres

device used in experiemnt: cpu
flow loss C library not being used as we are not on linux
data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: query
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: 16
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like

In [ ]:
!python3 main.py --config configs/config-seen-mse.yaml --alg mscn --result_dir ./results

In [20]:
!python3 main.py --config configs/config-seen-mse.yaml --alg mscn --result_dir ./results/seen/mscn/mse

data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: query
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: -1
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like_char_features: 0
  max_discrete_featurizing_buckets: 1
  max_like_featurizing_buckets: 1

In [ ]:
!python3 main.py --config configs/config-seen-flowloss.yaml --alg mscn --result_dir ./results/seen/mscn/flowloss

### Unseen template

In [18]:
!python3 main.py --config configs/config-unseen-postgres.yaml --alg true --result_dir ./results/unseen/true

data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: template
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: 16
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like_char_features: 0
  max_discrete_featurizing_buckets: 1
  max_like_featurizing_buckets

In [19]:
!python3 main.py --config configs/config-unseen-postgres.yaml --alg postgres --result_dir ./results/unseen/postgres

data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: template
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: 16
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like_char_features: 0
  max_discrete_featurizing_buckets: 1
  max_like_featurizing_buckets

In [21]:
!python3 main.py --config configs/config-unseen-flowloss.yaml --alg mscn --result_dir ./results/unseen/mscn/mse

data:
  bitmap_dir: ./queries/allbitmaps/
  diff_templates_seed: 1
  eval_query_dir: ./queries/ceb-imdb
  eval_templates: all
  no_regex_templates: 0
  num_samples_per_template: -1
  only_pgplan_data: 0
  query_dir: ./queries/ceb-imdb
  query_templates: all
  seed: 123
  skip7a: 1
  test_size: 0.4
  test_tmps: null
  train_test_split_kind: template
  train_tmps: null
  val_size: 0.2
db:
  db_host: localhost
  db_name: imdb
  port: 5432
  pwd: password
  user: imdb
eval:
  num_processes: -1
  save_pdf_plans: 0
  save_test_preds: 0
  use_wandb: 0
featurizer:
  bitmap_onehotmask: 1
  clamp_timeouts: 1
  embedding_fn: null
  embedding_pooling: null
  feat_mcvs: 0
  feat_onlyseen_maxy: 1
  feat_onlyseen_preds: 1
  feat_separate_alias: 0
  feat_separate_like_ests: 0
  global_feat_tables: 0
  global_features: 1
  heuristic_features: 1
  implied_pred_features: 0
  join_bitmap: 0
  join_features: onehot
  like_char_features: 0
  max_discrete_featurizing_buckets: 1
  max_like_featurizing_buckets

In [ ]:
!python3 main.py --config configs/config-unseen-flowloss.yaml --alg mscn --result_dir ./results/unseen/mscn/flowloss